In [1]:
import pandas as pd

quadruplexes = pd.read_csv("quadruplexes.txt", sep="\t", header=None)
quadruplexes.columns = ["seqid", "start", "end", "name", "score", "strand"]

gff = pd.read_csv("genomic.gff", sep="\t", comment='#', header=None)
gff.columns = ["seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"]

In [2]:
quadruplexes

,seqid,start,end,name,score,strand
0,NC_081552.1,255454,255479,G4_+,0,+
1,NC_081552.1,357388,357409,G4_+,0,+
2,NC_081552.1,392981,393000,G4_+,0,+
3,NC_081552.1,497424,497452,G4_+,0,+
4,NC_081552.1,516530,516554,G4_+,0,+
...,...,...,...,...,...,...
19425,NW_026739410.1,48301,48328,G4_+,0,+
19426,NW_026739413.1,15247,15274,G4_+,0,+
19427,NW_026739413.1,28565,28585,G4_+,0,+
19428,NW_026739413.1,29088,29111,G4_+,0,+


In [3]:
gff

,seqid,source,type,start,end,score,strand,phase,attributes
0,NC_081552.1,RefSeq,region,1,271964629,.,+,.,ID=NC_081552.1:1..271964629;Dbxref=taxon:35570...
1,NC_081552.1,Gnomon,gene,966,22098,.,+,.,ID=gene-LOC131997755;Dbxref=GeneID:131997755;N...
2,NC_081552.1,Gnomon,lnc_RNA,966,22098,.,+,.,ID=rna-XR_009398415.1;Parent=gene-LOC131997755...
3,NC_081552.1,Gnomon,exon,966,1124,.,+,.,ID=exon-XR_009398415.1-1;Parent=rna-XR_0093984...
4,NC_081552.1,Gnomon,exon,1185,3100,.,+,.,ID=exon-XR_009398415.1-2;Parent=rna-XR_0093984...
...,...,...,...,...,...,...,...,...,...
378417,NW_026739412.1,RefSeq,region,1,29435,.,+,.,ID=NW_026739412.1:1..29435;Dbxref=taxon:35570;...
378418,NW_026739413.1,RefSeq,region,1,29751,.,+,.,ID=NW_026739413.1:1..29751;Dbxref=taxon:35570;...
378419,NW_026739414.1,RefSeq,region,1,38434,.,+,.,ID=NW_026739414.1:1..38434;Dbxref=taxon:35570;...
378420,NW_026739415.1,RefSeq,region,1,26270,.,+,.,ID=NW_026739415.1:1..26270;Dbxref=taxon:35570;...


In [4]:
def annotate_feature(quad, feature_df, label):
    results = []
    for i, row in quad.iterrows():
        overlap = feature_df[
            (feature_df["seqid"] == row["seqid"]) &
            (feature_df["start"] <= row["end"]) &
            (feature_df["end"] >= row["start"])
        ]
        if not overlap.empty:
            results.append(label)
        else:
            results.append(None)
    return results

In [ ]:
quadruplexes["Exon"] = annotate_feature(quadruplexes, gff[gff["type"] == "exon"], "exon")
quadruplexes["Intron"] = annotate_feature(quadruplexes, gff[gff["type"] == "intron"], "intron")
quadruplexes["Gene"] = annotate_feature(quadruplexes, gff[gff["type"] == "gene"], "gene")

genes = gff[gff["type"] == "gene"].copy()
promoters = genes.copy()
promoters["start"], promoters["end"] = promoters["start"] - 1000, promoters["start"]

quadruplexes["Promoter"] = annotate_feature(quadruplexes, promoters, "promoter")

downstream = genes.copy()
downstream["start"], downstream["end"] = downstream["end"], downstream["end"] + 200
quadruplexes["Downstream"] = annotate_feature(quadruplexes, downstream, "downstream")

In [6]:
!pip install bioframe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 4.9 MB/s eta 0:00:00


In [7]:
import bioframe as bf

In [9]:
def calculate_introns(genes_df, exons_df):
    # Находим непокрытые экзонами участки генов
    cov = bf.coverage(genes_df, exons_df)
    intron_candidates = cov[cov['coverage'] < 1]

    # Объединяем смежные участки
    if not intron_candidates.empty:
        return bf.merge(intron_candidates[['chrom', 'start', 'end']])
    return pd.DataFrame(columns=['chrom', 'start', 'end'])

# Преобразуем данные
genes_df = genes[["seqid", "start", "end"]].rename(columns={"seqid": "chrom"})
exons_df = exons[["seqid", "start", "end"]].rename(columns={"seqid": "chrom"})

# Вычисляем интроны
introns = calculate_introns(genes_df, exons_df)
introns = introns.rename(columns={"chrom": "seqid"})

# Проверка перед аннотацией
if not introns.empty:
    quadruplexes["Intron"] = annotate_feature(quadruplexes, introns, "intron")
else:
    print("Предупреждение: Не найдено ни одного интрона!")
    quadruplexes["Intron"] = None


quadruplexes["Exon"] = annotate_feature(quadruplexes, gff[gff["type"] == "exon"], "exon")
quadruplexes["Gene"] = annotate_feature(quadruplexes, gff[gff["type"] == "gene"], "gene")

# Вычисляем интроны как гены минус экзоны
genes = gff[gff["type"] == "gene"].copy()
exons = gff[gff["type"] == "exon"].copy()

# Преобразуем в формат для bioframe
genes_df = genes[["seqid", "start", "end"]].rename(columns={"seqid": "chrom"})
exons_df = exons[["seqid", "start", "end"]].rename(columns={"seqid": "chrom"})

# Находим интроны
#introns = bf.setdiff(genes_df, exons_df)
#introns = introns.rename(columns={"chrom": "seqid"})

# Аннотируем интроны
#quadruplexes["Intron"] = annotate_feature(quadruplexes, introns, "intron")

# Аннотируем промоторы и downstream
genes = gff[gff["type"] == "gene"].copy()
promoters = genes.copy()
promoters["start"], promoters["end"] = promoters["start"] - 1000, promoters["start"]
quadruplexes["Promoter"] = annotate_feature(quadruplexes, promoters, "promoter")

downstream = genes.copy()
downstream["start"], downstream["end"] = downstream["end"], downstream["end"] + 200
quadruplexes["Downstream"] = annotate_feature(quadruplexes, downstream, "downstream")

def count_and_fraction(df, col):
    count = df[col].notna().sum()
    fraction = count / len(df)
    return count, round(fraction, 3)

stats = {
    "Exons": count_and_fraction(quadruplexes, "Exon"),
    "Introns": count_and_fraction(quadruplexes, "Intron"),
    "Promoters": count_and_fraction(quadruplexes, "Promoter"),
    "Downstream": count_and_fraction(quadruplexes, "Downstream"),
    "Intergenic": (len(quadruplexes) - sum(quadruplexes[["Exon", "Intron", "Gene"]].notna().any(axis=1)),
                  round((len(quadruplexes) - sum(quadruplexes[["Exon", "Intron", "Gene"]].notna().any(axis=1))) / len(quadruplexes), 3))
}

table = pd.DataFrame(stats, index=["Count", "Fraction"])
table

Предупреждение: Не найдено ни одного интрона!


,Exons,Introns,Promoters,Downstream,Intergenic
Count,504.000,0.0,279.000,89.000,9545.000
Fraction,0.026,0.0,0.014,0.005,0.491


In [ ]:
!apt-get install -y bedtools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  bedtools
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 563 kB of archives.
After this operation, 1,548 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 bedtools amd64 2.30.0+dfsg-2ubuntu0.1 [563 kB]
Fetched 563 kB in 1s (919 kB/s)
Selecting previously unselected package bedtools.
(Reading database ... 126319 files and directories currently installed.)
Preparing to unpack .../bedtools_2.30.0+dfsg-2ubuntu0.1_amd64.deb ...
Unpacking bedtools (2.30.0+dfsg-2ubuntu0.1) ...
Setting up bedtools (2.30.0+dfsg-2ubuntu0.1) ...


In [ ]:
import pandas as pd

gff = pd.read_csv("genomic.gff", sep="\t", comment="#", header=None,
                  names=["seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"])
quadruplexes = pd.read_csv("quadruplexes.txt", sep="\t", header=None,
                           names=["seqid", "start", "end", "name", "score", "strand"])


def count_regions_with_quad(features_df, quads_df):
    features_with_quad = features_df.apply(
        lambda row: ((quads_df["start"] < row["end"]) & (quads_df["end"] > row["start"]) &
                     (quads_df["seqid"] == row["seqid"])).any(), axis=1)
    count = features_with_quad.sum()
    fraction = round(count / len(features_df), 3) if len(features_df) > 0 else 0
    return count, fraction

exons = gff[gff["type"] == "exon"]
genes = gff[gff["type"] == "gene"].copy()

promoters = genes.copy()
promoters["end"] = promoters["start"]
promoters["start"] = promoters["start"] - 1000

downstream = genes.copy()
downstream["start"] = downstream["end"]
downstream["end"] = downstream["end"] + 200

results = {
    "Exons": count_regions_with_quad(exons, quadruplexes),
    "Promoters (1000 up from TSS)": count_regions_with_quad(promoters, quadruplexes),
    "Downstream (200 bp)": count_regions_with_quad(downstream, quadruplexes),
}

genes["region_id"] = genes.index
quad_in_gene = genes.apply(
    lambda row: ((quadruplexes["start"] < row["end"]) & (quadruplexes["end"] > row["start"]) &
                 (quadruplexes["seqid"] == row["seqid"])).any(), axis=1)
genes_with_quad = quad_in_gene.sum()

total_regions = len(genes)
intergenic_count = total_regions - genes_with_quad
intergenic_fraction = round(intergenic_count / total_regions, 3)

results["Introns"] = (0, 0.0)
results["Intergenic"] = (intergenic_count, intergenic_fraction)

table2 = pd.DataFrame(results, index=["Число участков с квадруплексом", "Доля участков с квадруплексом"]).T
print(table2)


                              Число участков с квадруплексом  \
Exons                                                 1162.0   
Promoters (1000 up from TSS)                           265.0   
Downstream (200 bp)                                     86.0   
Introns                                                  0.0   
Intergenic                                           15249.0   

                              Доля участков с квадруплексом  
Exons                                                 0.006  
Promoters (1000 up from TSS)                          0.014  
Downstream (200 bp)                                   0.005  
Introns                                               0.000  
Intergenic                                            0.832  
